In [1]:
import praw
import requests
import pandas as pd
import json



from config import reddit_client_id
from config import reddit_secret_key
from config import reddit_user_name
from config import reddit_password
from config import yahoo_api_key


'''
auth = requests.auth.HTTPBasicAuth(reddit_client_id,reddit_secret_key)
data = {
    'grant_type': 'password',
    'username': reddit_user_name,
    'password': reddit_password
}
headers = {'User-Agent': 'redditapi/1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token12 = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {token12}'}}


url = "https://oauth.reddit.com/r/wallstreetbets/"
query_url = url + "best"
best_posts = requests.get(query_url, headers=headers)
best_json = best_posts.json()
best_json

'''

{'kind': 'Listing',
 'data': {'after': 't3_18fxw41',
  'dist': 27,
  'modhash': None,
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'wallstreetbets',
     'selftext': "Join [WSB's community voice chat](https://discord.com/channels/1037784194476163223/1083096862099263528), every day from 8:30am to whenever!\n\nCheck out our [Earnings Thread](https://reddit.com/r/wallstreetbets/about/sticky?num=2) and [Rules](https://www.reddit.com/r/wallstreetbets/wiki/contentguide).",
     'author_fullname': 't2_bd6q5',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'Daily Discussion Thread for December 11, 2023',
     'link_flair_richtext': [{'e': 'text', 't': 'Daily Discussion'}],
     'subreddit_name_prefixed': 'r/wallstreetbets',
     'hidden': False,
     'pwls': 7,
     'link_flair_css_class': 'daily',
     'downs': 0,
     'thumbnail_height': None,
     'top_awarded_type': No

In [31]:

#yahoo_base_url = "https://yahoo-finance127.p.rapidapi.com/price/tsla"
yahoo_base_url = "https://yahoo-finance127.p.rapidapi.com/price/"


headers = {
    "X-RapidAPI-Key": yahoo_api_key,
    "X-RapidAPI-Host": "yahoo-finance127.p.rapidapi.com"
}
# response = requests.get(yahoo_base_url, headers=headers)

company_names = []
stock_symbol = str(input("Enter stock symbol to search"))


#------------------------------------------------------------------------TO-DO - Add error handling for when ticker is not found

# token1 = str(response.json())
#json.dumps(token1, indent=4, sort_keys=True)
yahoo_response = requests.get(yahoo_base_url + stock_symbol, headers=headers).json()
#yahoo_response
# print(json.dumps(data, indent=4, sort_keys=True))

#Add longname as search term for reddit
company_names.append(yahoo_response["longName"])

#Add stock symbol as search term for reddit
company_names.append(stock_symbol)

#Get alternate company names to use for reddit search
msg = f"{stock_symbol}'s LongName is {company_names[0]}.  Add another search term and press Enter, or press Cancel when done'"

while True:
    response = str(input(msg))
    if len(response)> 0:
        company_names.append(response)
    else:
        break

#remove duplicates from company_names
reddit_search_list = pd.Series(company_names).drop_duplicates().tolist()

['Tesla, Inc.', 'tsla', 'Tesla', 'Frank', 'Frank']


['Tesla, Inc.', 'tsla', 'Tesla', 'Frank']


In [39]:

import requests
import praw
auth = requests.auth.HTTPBasicAuth(reddit_client_id,reddit_secret_key)
data = {
    'grant_type': 'password',
    'username': reddit_user_name,
    'password': reddit_password
}
headers = {'User-Agent': 'redditapi/1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token12 = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {token12}'}}
headers
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

reddit_posts = []
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_secret_key,
    password = data['password'],
    user_agent=f"testscript by u/{data['username']}",
    username = data['username'],
)


#-----------------------------------------------------------TO-DO: Do we need to set a different limit or search criteria?
for term in reddit_search_list:
    for submission in reddit.subreddit("wallstreetbets").search(term, limit=10, sort="new"):

        reddit_posts.append({"search term": term,
                             "title": submission.title,
                            "upvotes": submission.score,
                            "num_comments": submission.num_comments,
                            "date_utc": submission.created_utc})
    


reddit_df = pd.DataFrame(reddit_posts)
reddit_df.drop_duplicates(subset=["title"],keep="first",inplace=True)
reddit_df

,search term,title,upvotes,num_comments,date_utc
0,"Tesla, Inc.",JPOW looking at job data today,6,1,1.702045e+09
1,"Tesla, Inc.",Where is all the damn gain porn?,33,38,1.701983e+09
2,"Tesla, Inc.",TSLA Institutional Ownership Over Time.,10,5,1.701189e+09
3,"Tesla, Inc.",Tesla insiders had perfect timing with selling...,97,27,1.700485e+09
4,"Tesla, Inc.",$1.2k —> $10.7k thanks to $TSLA,522,69,1.700415e+09
5,"Tesla, Inc.",US Senator casually buying leveraged positions...,11258,651,1.700247e+09
6,"Tesla, Inc.",I know how much yall like technicals (TSLA),7,22,1.699033e+09
7,"Tesla, Inc.","Simple, just swing shares of tsla when RSI hit...",67,28,1.698942e+09
8,"Tesla, Inc.",Huge spikes on Tesla chart today - is it anoth...,9,15,1.698434e+09
9,"Tesla, Inc.",Tesla's AI can increase its valuation equal of...,23,211,1.698424e+09


In [11]:
yahoo_finance = []

yahoo_finance.append({"Symbol": yahoo_response["symbol"],
                        "Day_Range_raw": yahoo_response["regularMarketDayRange"]["raw"],
                        "Market_Price": yahoo_response["regularMarketPrice"]["raw"],
                        "Company_Name": yahoo_response["longName"]})

yahoo_finance_df = pd.DataFrame(yahoo_finance)
yahoo_finance_df.head()


,Symbol,Day_Range_raw,Market_Price,Company_Name
0,TSLA,236.98 - 244.08,242.64,"Tesla, Inc."
